# Objectives {.unnumbered}

By the end of this lab, you will:
1. Load and analyze the **Lightcast dataset** in **Spark DataFrame**.
2. Create **five easy and three medium-complexity visualizations** using **Plotly**.
3. Explore **salary distributions, employment trends, and job postings**.
4. Analyze **skills in relation to NAICS/SOC/ONET codes and salaries**.
5. Customize **colors, fonts, and styles** in all visualizations (**default themes result in a 2.5-point deduction**).
6. Follow **best practices for reporting on data communication**.

# Step 1: Load the Dataset {.unnumbered}


In [43]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import ast
import plotly.graph_objects as go

# Initialize Spark Session
spark = SparkSession.builder.appName("Lightcast_job_postings").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

# Show Schema and Sample Data
df.printSchema()
df.show(5)


root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: string (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: string (nullable = true)
 |-- EXPIRED: string (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: string (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVELS: integer (

# Salary Distribution by Employment Type
- Identify salary trends across different employment types.
- **Filter the dataset**
  - Remove records where **salary is missing or zero**.
- **Aggregate Data**
  - Group by **employment type** and compute salary distribution.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `EMPLOYMENT_TYPE_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize **colors, fonts, and styles** to avoid a **2.5-point deduction**.
- **Explanation:** Write two sentences about what the graph reveals.



In [26]:
# 选择需要的列，并去除缺失或为 0 的数据
pdf = df.select("EMPLOYMENT_TYPE_NAME", "SALARY_FROM").toPandas()
pdf = pdf.dropna(subset=['SALARY_FROM'])
pdf = pdf[pdf['SALARY_FROM'] > 0]

# 修复乱码问题
pdf['EMPLOYMENT_TYPE_NAME'] = pdf['EMPLOYMENT_TYPE_NAME'].str.encode('ascii', 'ignore').str.decode('ascii')

fig = px.box(
      pdf,
    x="EMPLOYMENT_TYPE_NAME",
    y="SALARY_FROM",
    title="Salary Distribution by Employment Type",
    color_discrete_sequence=["#0ABAB5"]
)
fig.update_layout(font_family="Arial", title_font_size=20)
fig.show()


# Salary Distribution by Industry
- Compare salary variations across industries.
- **Filter the dataset**
  - Keep records where **salary is greater than zero**.
- **Aggregate Data**
  - Group by **NAICS industry codes**.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize colors, fonts, and styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [27]:
# Filter the dataset: keep only records where SALARY_FROM > 0
pdf_industry = df.select("NAICS2_NAME", "SALARY_FROM").toPandas()
pdf_industry = pdf_industry.dropna(subset=['SALARY_FROM', 'NAICS2_NAME'])
pdf_industry = pdf_industry[pdf_industry['SALARY_FROM'] > 0]

# 清理 NAICS2_NAME 字段避免乱码
pdf_industry['NAICS2_NAME'] = pdf_industry['NAICS2_NAME'].str.encode('ascii', 'ignore').str.decode('ascii')

fig_industry = px.box(
    pdf_industry,
    x="NAICS2_NAME",
    y="SALARY_FROM",
    title="Salary Distribution by Industry",
    color_discrete_sequence=["#A1E8AF"]
)
fig_industry.update_layout(
    font_family="Arial",
    title_font_size=16,
    xaxis=dict(
        tickangle=-45,
        tickfont=dict(size=10)
    ),
    yaxis=dict(
        tickfont=dict(size=10)
    )
)
fig_industry.show()


# Job Posting Trends Over Time
- Analyze how job postings fluctuate over time.
- **Aggregate Data**
  - Count job postings per **posted date (`POSTED`)**.
- **Visualize results**
  - Create a **line chart** where:
    - **X-axis** = `POSTED`
    - **Y-axis** = `Number of Job Postings`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [28]:
pdf_trend = df.select("POSTED").toPandas()
pdf_trend = pdf_trend.dropna(subset=['POSTED'])

# 将 POSTED 列转换为日期格式
pdf_trend['POSTED'] = pd.to_datetime(pdf_trend['POSTED'])

# 按日期统计职位发布数量
job_trend = pdf_trend.groupby('POSTED').size().reset_index(name='Job_Postings')

# 按日期排序
job_trend = job_trend.sort_values('POSTED')

# 绘制折线图
fig_trend = px.line(
    job_trend,
    x='POSTED',
    y='Job_Postings',
    title='Job Posting Trends Over Time',
    markers=True,
    line_shape='linear'
)

# 设置颜色和字体样式
fig_trend.update_traces(line=dict(color='#0ABAB5'))  # 蒂芙尼蓝
fig_trend.update_layout(
    font_family="Arial",
    title_font_size=18,
    xaxis_title='Date Posted',
    yaxis_title='Number of Job Postings',
    xaxis=dict(tickfont=dict(size=12)),
    yaxis=dict(tickfont=dict(size=12))
)

fig_trend.show()

# Top 10 Job Titles by Count
- Identify the most frequently posted job titles.
- **Aggregate Data**
  - Count the occurrences of each **job title (`TITLE_NAME`)**.
  - Select the **top 10 most frequent titles**.
- **Visualize results**
  - Create a **bar chart** where:
    - **X-axis** = `TITLE_NAME`
    - **Y-axis** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [29]:
pdf_titles = df.select("TITLE_NAME").toPandas()
pdf_titles = pdf_titles.dropna(subset=['TITLE_NAME'])

# 按职位名称统计数量
title_counts = pdf_titles['TITLE_NAME'].value_counts().nlargest(10).reset_index()
title_counts.columns = ['TITLE_NAME', 'Job_Count']

# 绘制条形图
fig_titles = px.bar(
    title_counts,
    x='TITLE_NAME',
    y='Job_Count',
    title='Top 10 Job Titles by Count',
    color_discrete_sequence=['#0ABAB5']  # 蒂芙尼蓝
)

# 自定义字体、轴标签和样式
fig_titles.update_layout(
    font_family="Arial",
    title_font_size=18,
    xaxis_title='Job Title',
    yaxis_title='Job Count',
    xaxis=dict(tickfont=dict(size=12), tickangle=-30),
    yaxis=dict(tickfont=dict(size=12))
)

fig_titles.show()

# Remote vs On-Site Job Postings
- Compare the proportion of remote and on-site job postings.
- **Aggregate Data**
  - Count job postings by **remote type (`REMOTE_TYPE_NAME`)**.
- **Visualize results**
  - Create a **pie chart** where:
    - **Labels** = `REMOTE_TYPE_NAME`
    - **Values** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [30]:
pdf_remote = df.select("REMOTE_TYPE_NAME").toPandas()
pdf_remote = pdf_remote.dropna(subset=['REMOTE_TYPE_NAME'])
pdf_remote['REMOTE_TYPE_NAME'] = pdf_remote['REMOTE_TYPE_NAME'].replace('[None]', 'On-site')

# 按远程类型统计数量
remote_counts = pdf_remote['REMOTE_TYPE_NAME'].value_counts().reset_index()
remote_counts.columns = ['REMOTE_TYPE_NAME', 'Job_Count']

print(pdf_remote.head(20))

# 绘制饼图
fig_remote = px.pie(
    remote_counts,
    names='REMOTE_TYPE_NAME',
    values='Job_Count',
    title='Remote vs On-Site Job Postings',
    color_discrete_sequence=['#0ABAB5', '#FFA07A', '#FECB52']  # 蒂芙尼蓝+橙粉+黄色
)

# 设置字体和样式
fig_remote.update_layout(
    font_family="Arial",
    title_font_size=18
)

fig_remote.show()

   REMOTE_TYPE_NAME
0           On-site
1            Remote
2           On-site
3           On-site
4           On-site
5            Remote
6           On-site
7           On-site
8           On-site
9           On-site
10       Not Remote
11          On-site
12          On-site
13          On-site
14           Remote
15           Remote
16          On-site
17           Remote
18          On-site
19          On-site


# Skill Demand Analysis by Industry (Stacked Bar Chart)
- Identify which skills are most in demand in various industries.
- **Aggregate Data**
  - Extract **skills** from job postings.
  - Count occurrences of skills grouped by **NAICS industry codes**.
- **Visualize results**
  - Create a **stacked bar chart** where:
    - **X-axis** = `Industry`
    - **Y-axis** = `Skill Count`
    - **Color** = `Skill`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [35]:
pdf_skills = df.select("NAICS2_NAME", "SKILLS_NAME").toPandas()

# Step 2: Drop rows with missing values
pdf_skills = pdf_skills.dropna(subset=['NAICS2_NAME', 'SKILLS_NAME'])

# Step 3: Convert SKILLS_NAME string representation of list into actual Python lists
def parse_skills(x):
    try:
        return ast.literal_eval(x)
    except:
        return [x.strip()]

pdf_skills['SKILLS_LIST'] = pdf_skills['SKILLS_NAME'].apply(parse_skills)

# Step 4: Explode the list to create one row per skill
pdf_skills_exploded = pdf_skills.explode('SKILLS_LIST')
pdf_skills_exploded['SKILLS_LIST'] = pdf_skills_exploded['SKILLS_LIST'].str.strip()

# Step 5: Group and count skill occurrences per industry
skill_counts = pdf_skills_exploded.groupby(['NAICS2_NAME', 'SKILLS_LIST']).size().reset_index(name='Skill_Count')

# Step 6: Select top 5 most common skills overall
top_skills = skill_counts.groupby('SKILLS_LIST')['Skill_Count'].sum().nlargest(5).index.tolist()
skill_counts_top = skill_counts[skill_counts['SKILLS_LIST'].isin(top_skills)]

# Step 7: Plot stacked bar chart
fig_skills = px.bar(
    skill_counts_top,
    x='NAICS2_NAME',
    y='Skill_Count',
    color='SKILLS_LIST',
    title='Top 5 In-Demand Skills by Industry',
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig_skills.update_layout(
    font_family="Arial",
    title_font_size=18,
    xaxis_title='Industry',
    yaxis_title='Skill Count',
    xaxis=dict(tickfont=dict(size=10), tickangle=-45),
    yaxis=dict(tickfont=dict(size=10))
)

fig_skills.show()


# Salary Analysis by ONET Occupation Type (Bubble Chart)
- Analyze how salaries differ across ONET occupation types.
- **Aggregate Data**
  - Compute **median salary** for each occupation in the **ONET taxonomy**.
- **Visualize results**
  - Create a **bubble chart** where:
    - **X-axis** = `ONET_NAME`
    - **Y-axis** = `Median Salary`
    - **Size** = Number of job postings
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [46]:
pdf_onet = df.select("ONET_NAME", "SALARY_FROM").toPandas()

# Step 2: 按 ONET_NAME 分组计算中位数薪资与职位数量
onet_salary_stats = pdf_onet.groupby('ONET_NAME').agg(
    Median_Salary=('SALARY_FROM', 'median'),
    Job_Postings=('SALARY_FROM', 'count')
).reset_index()

# Step 3: 绘制气泡图
fig_onet_bubble = px.scatter(
    onet_salary_stats,
    x='ONET_NAME',
    y='Median_Salary',
    size='Job_Postings',
    title='Salary Analysis by ONET Occupation Type',
    color_discrete_sequence=['#0ABAB5'],  # Tiffany Blue
    size_max=50
)

fig_onet_bubble.update_layout(
    font_family="Arial",
    title_font_size=18,
    xaxis_title='ONET Occupation',
    yaxis_title='Median Salary',
    xaxis=dict(tickfont=dict(size=10)),
    yaxis=dict(tickfont=dict(size=12))
)

fig_onet_bubble.show()

# Career Pathway Trends (Sankey Diagram)
- Visualize job transitions between different occupation levels.
- **Aggregate Data**
  - Identify career transitions between **SOC job classifications**.
- **Visualize results**
  - Create a **Sankey diagram** where:
    - **Source** = `SOC_2021_2_NAME`
    - **Target** = `SOC_2021_3_NAME`
    - **Value** = Number of transitions
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [50]:
pdf_career = df.select("SOC_2021_2_NAME", "SOC_2021_3_NAME").toPandas()

# Step 3: 统计职业路径转移
career_flows = pdf_career.groupby(['SOC_2021_2_NAME', 'SOC_2021_3_NAME']).size().reset_index(name='Count')

# Step 4: 创建节点和索引映射
all_nodes = list(pd.concat([career_flows['SOC_2021_2_NAME'], career_flows['SOC_2021_3_NAME']]).unique())
node_indices = {node: i for i, node in enumerate(all_nodes)}

# Step 5: 准备 Sankey 图所需数据
source_indices = career_flows['SOC_2021_2_NAME'].map(node_indices)
target_indices = career_flows['SOC_2021_3_NAME'].map(node_indices)

# Step 6: 绘制 Sankey 图
fig_sankey = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_nodes,
        color="lightblue"
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=career_flows['Count'],
        color="rgba(10,186,181,0.4)"  # Tiffany Blue with transparency
    )
)])

fig_sankey.update_layout(
    title_text="Career Pathway Trends (SOC 2 to SOC 3)",
    font=dict(size=12, family="Arial")
)

fig_sankey.show()

career_flows['SOC_2021_2_NAME'].value_counts()

SOC_2021_2_NAME
Computer and Mathematical Occupations    1
Name: count, dtype: int64